# MTH793P - Coursework 1

This is a template notebook for the computational exercises of Coursework #1 of the module MTH793P, Advanced Machine Learning. Closely follow the instructions in this template in order to complete the assessment and to obtain full marks. Please only modify cells where you are instructed to do so. Failure to comply may result in unexpected errors that can lead to mark deductions.

As usual, we begin by loading the necessary libraries.

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse as sp
# from scipy.sparse import linalg

Create two lists *nodes* and *edges* and one NumPy array *weights*. The list *nodes* should contain all names of the nodes in the graph of Coursework 1, Question 1, in alphabetical order. The list edges should include lists that contain the indices of the nodes that are connected by the individual edge. For example, the first edge connects node 'Batman' and node 'Jessica Jones', so the list for this edge should be [0, 3], as 'Batman' is the first entry of the list *nodes* and 'Jessica Jones' is the fourth entry.  The Numpy array *weights* should contain the individual weights assigned to each edge.

In [10]:
# YOUR CODE HERE
nodes = ['Batman', 'Catwoman', 'Deadpool', 'Jessica Jones', 'Spiderman', 'Wonder woman']
edges = [[0,3], [0,1], [0,2], [0,5], [2,3], [3,5], [3,4], [1,5], [1,4], [2,4], [4,5]]
weights = np.array([4, 81, 16, 64, 64, 36, 49, 49, 1, 49, 1])

Display your lists and array with the following cell.

In [11]:
print('We consider a graph with nodes/vertices {n}, edges {e} and weights {w}.'.format( \
                            n = nodes, e = edges, w = weights))

We consider a graph with nodes/vertices ['Batman', 'Catwoman', 'Deadpool', 'Jessica Jones', 'Spiderman', 'Wonder woman'], edges [[0, 3], [0, 1], [0, 2], [0, 5], [2, 3], [3, 5], [3, 4], [1, 5], [1, 4], [2, 4], [4, 5]] and weights [ 4 81 16 64 64 36 49 49  1 49  1].


Write a function **construct_incidence_matrix** that takes the lists *nodes*, *edges* and the NumPy array *weights* as arguments. The function should return the incidence matrix *incidence_matrix* that corresponds to the weighted graph. The construction of the incidence matrix is described in the lecture notes.

In [12]:
def construct_incidence_matrix(nodes, edges, weights):
    # YOUR CODE HERE
    matrix = np.zeros(shape=(len(edges),len(nodes)))
    for i, row in enumerate(matrix):
        row[edges[i][0]] = - (weights[i])**0.5
        row[edges[i][1]] = (weights[i])**0.5
    return matrix

You can test your function with the following cell.

In [14]:
from numpy.testing import assert_array_equal
incidence_matrix = construct_incidence_matrix(nodes, edges, weights)    
print('The incidence matrix of our graph is \n {i}.'.format(i = incidence_matrix.astype(int)))

# The following code is testing the previous code against a specific example    
test_nodes = ['Batman', 'Catwoman', 'Spiderman']
test_edges = [[0, 1], [1, 2]]
test_weights = np.array([81, 1])
test_incidence_matrix = construct_incidence_matrix(test_nodes, test_edges, test_weights)

assert_array_equal(test_incidence_matrix.astype(int), np.array([[-9, 9, 0],[0, -1, 1]]))


The incidence matrix of our graph is 
 [[-2  0  0  2  0  0]
 [-9  9  0  0  0  0]
 [-4  0  4  0  0  0]
 [-8  0  0  0  0  8]
 [ 0  0 -8  8  0  0]
 [ 0  0  0 -6  0  6]
 [ 0  0  0 -7  7  0]
 [ 0 -7  0  0  0  7]
 [ 0 -1  0  0  1  0]
 [ 0  0 -7  0  7  0]
 [ 0  0  0  0 -1  1]].


Next, compute the corresponding graph-Laplacian for the incidence matrix *incidence_matrix* from the previous exercise and store it in a variable named *graph_laplacian*. Follow the definition from the lecture notes.

In [15]:
# YOUR CODE HERE
graph_laplacian = incidence_matrix.T @ incidence_matrix

You can test your function with the following cell.

In [16]:
print('The graph Laplacian of our graph is \n {g}.'.format(g = graph_laplacian.astype(int)))

The graph Laplacian of our graph is 
 [[165 -81 -16  -4   0 -64]
 [-81 131   0   0  -1 -49]
 [-16   0 129 -64 -49   0]
 [ -4   0 -64 153 -49 -36]
 [  0  -1 -49 -49 100  -1]
 [-64 -49   0 -36  -1 150]].


We want to use our graph to determine whether a node in the graph belongs to the class "Marvel" or the class "DC". Suppose we are in a semi-supervised setting, where the node "Deadpool" is already labelled $v_{\text{Deadpool}} = 0$ (class "Marvel") and the node "Catwoman" is labelled as $v_{\text{Catwoman}} = 1$ (class "DC"). Here $v$ is the mathematical notation of the label vector. We follow the instructions in the lecture notes and formulate this as a linear system. We can either define appropriate projection matrices or create sub-matrices from the graph-Laplacian by choosing the correct indices. How you set up the linear system is up to you. Store your linear system in a variable named *linear_system* and the right-hand-side in a variable *right_hand_side*.

In [17]:
# YOUR CODE HERE
labeled = [1,2]
unlabeled = [0,3,4,5]
y = np.array([1,0]).reshape(2,1)
for i, row in enumerate(graph_laplacian[unlabeled]):
    if i == 0:
        linear_system = row[unlabeled]
        right_hand_side = row[labeled]
    else:
        linear_system = np.vstack((linear_system, row[unlabeled]))
        right_hand_side = np.vstack((right_hand_side, row[labeled]))

Solve your linear system and store your labels in an array named *remaining_labels*. Create also an array *all_labels* that contains all labels, as well as a boolean array *thresholded_labels* of the same size as *all_labels* with True or False values. An entry should be true if the corresponding entry in *all_labels* is larger than 0.5 and false otherwise.

In [22]:
# YOUR CODE HERE
A = linear_system
b = right_hand_side@y

remaining_labels = np.absolute(np.linalg.solve(A, b)) # f_tilda
all_labels = np.vstack((remaining_labels[0],y))
all_labels = np.vstack((all_labels,remaining_labels[1:]))
thresholded_labels = np.round(all_labels,0).astype(bool)
print(thresholded_labels)

[[ True]
 [ True]
 [False]
 [False]
 [False]
 [ True]]


Check your results with the following cell.

In [23]:
print('The computed labels are {a}. Setting all values above 0.5 to one and'.format( \
        a = all_labels), 'the remaining ones to zero yields {t}.'.format(t = \
        thresholded_labels.astype(int)))

The computed labels are [[0.77273292]
 [1.        ]
 [0.        ]
 [0.22924953]
 [0.12945476]
 [0.71224896]]. Setting all values above 0.5 to one and the remaining ones to zero yields [[1]
 [1]
 [0]
 [0]
 [0]
 [1]].


We conclude this coursework by computing the second eigenvector, i.e. the eigenvector that corresponds to the second smallest eigenvalue, of the graph-Laplacian *graph_laplacian*. Explore how to compute eigenvalues and eigenvectors, in particular of sparse matrices if you have used them.

In [24]:
# YOUR CODE HERE
eigenvalues, eigenvectors = np.linalg.eig(graph_laplacian)

Display the second smallest eigenvalue and the corresponding eigenvector. What do you observe?

In [25]:
print('The second smallest eigenvalue of the graph Laplacian is {eval}.'.format(eval = \
        eigenvalues[1].real), 'The corresponding eigenvector is {evec}.T.'.format(evec = \
        eigenvectors[:, 1].T.real))

The second smallest eigenvalue of the graph Laplacian is 33.16375166078198. The corresponding eigenvector is [ 0.39722732  0.48514106 -0.39542065 -0.30661694 -0.50260786  0.32227706].T.
